In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import numpy as np
from tqdm import tqdm
import pandas as pd
import os
from io import StringIO

In [2]:
data_folder = "../data"
winners = pd.read_csv(os.path.join(data_folder, "winners.csv")).drop(columns="Unnamed: 0")

In [3]:
len(os.listdir(data_folder))

1

In [4]:
driver = webdriver.Firefox()
driver.maximize_window()
errors = ""

for idx, row in tqdm(winners.iterrows()):
        try:
                winner_folder = os.path.join(data_folder, row["unique_id"])
                os.mkdir(winner_folder)
                driver.get(row["link"])

                # Make table with description
                info = {}
                header = driver.find_element(By.CLASS_NAME, "w3-half").text.split("\n")
                details = driver.find_elements(By.CLASS_NAME, "w3-third")[1:4]
                assets_liabilities = details[1].text.split("\n")
                info.update({"constituency": [header[1]]})
                info.update({
                        "party": [header[2].split(":")[-1].strip()],
                        "so_do_wo": [header[3].split(":")[-1].strip()],
                        "age": [header[4].split(":")[-1].strip()],
                        "voter_in": [header[5].split(":")[-1].strip()],
                        "profession": [header[6].split(":")[-1].strip()],
                        "spouse_profession": [header[7].split(":")[-1].strip()],
                        "n_crime_cases": [details[0].text.split(":")[-1].strip().replace("\n", " ")],
                        "assets": [assets_liabilities[1].split(":")[-1].strip()],
                        "liabilities": [assets_liabilities[2].split(":")[-1].strip()]
                })
                pd.DataFrame(info).to_csv(os.path.join(winner_folder, "info.csv"))
                pd.read_html(StringIO(driver.find_element(By.ID, "income_tax").get_attribute("outerHTML")))[0].to_csv(os.path.join(winner_folder, "tax_returns.csv"))
                pd.read_html(StringIO(driver.find_element(By.ID, "movable_assets").get_attribute("outerHTML")))[0].to_csv(os.path.join(winner_folder, "movable_assets.csv"))
                pd.read_html(StringIO(driver.find_element(By.ID, "immovable_assets").get_attribute("outerHTML")))[0].to_csv(os.path.join(winner_folder, "immovable_assets.csv"))
                pd.read_html(StringIO(driver.find_element(By.ID, "liabilities").get_attribute("outerHTML")))[0].to_csv(os.path.join(winner_folder, "liabilities.csv"))
        except IndexError as e:
                print(e)
                errors += row["unique_id"]

2551it [58:54,  1.49s/it]

list index out of range


4084it [1:32:40,  1.36s/it]


In [7]:
with open(os.path.join(data_folder, "errors.txt"), "w") as f:
        f.write(errors + "\n")

In [5]:
errors

'Puducherry2021_14'